# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [24]:
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore). Don't forget to enrich the table with the 'dayofweek' column from the previous day's .csv-file.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [4]:
df_unscaled = pd.read_csv('../data/day-of-week-not-scaled.csv')
df_original = pd.read_csv('../data/dayofweek.csv')
df_unscaled['dayofweek'] = df_original['dayofweek']

In [5]:
X = df_unscaled.drop('dayofweek', axis=1)
y = df_unscaled['dayofweek']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=21, stratify=y
)

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [6]:
param_grid_svm = {
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'C': [0.01, 0.1, 1, 1.5, 5, 10],
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced', None]
}

In [7]:
svm_grid = GridSearchCV(
    SVC(random_state=21, probability=True), 
    param_grid_svm, 
    scoring='accuracy', 
    n_jobs=-1, 
    cv=5       
)

In [8]:
svm_grid.fit(X_train, y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",SVC(probabili...ndom_state=21)
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'C': [0.01, 0.1, ...], 'class_weight': ['balanced', None], 'gamma': ['scale', 'auto'], 'kernel': ['linear', 'rbf', ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and param

In [9]:
svm_results = pd.DataFrame(svm_grid.cv_results_)
svm_results = svm_results.sort_values('rank_test_score').filter(
    ['params', 'mean_test_score', 'std_test_score', 'rank_test_score']
)


In [10]:
svm_results.head()

,params,mean_test_score,std_test_score,rank_test_score
70,"{'C': 10, 'class_weight': None, 'gamma': 'auto...",0.876109,0.018419,1
64,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.863500,0.010870,2
58,"{'C': 5, 'class_weight': None, 'gamma': 'auto'...",0.816018,0.008116,3
52,"{'C': 5, 'class_weight': 'balanced', 'gamma': ...",0.808608,0.021007,4
60,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.721052,0.034438,5


## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [11]:
param_grid_tree = {
    'max_depth': list(range(1, 50)),
    'class_weight': ['balanced', None],
    'criterion': ['entropy', 'gini']
}

In [12]:
tree_grid = GridSearchCV(
    DecisionTreeClassifier(random_state=21),
    param_grid_tree,
    scoring='accuracy',
    n_jobs=-1,
    cv=5
)

In [13]:
tree_grid.fit(X_train, y_train)


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",DecisionTreeC...ndom_state=21)
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'class_weight': ['balanced', None], 'criterion': ['entropy', 'gini'], 'max_depth': [1, 2, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is display

In [14]:
print(f"Best parameters: {tree_grid.best_params_}")
print(f"Best accuracy: {tree_grid.best_score_:.5f}")

Best parameters: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 22}
Best accuracy: 0.87312


## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [15]:
param_grid_rf = {
    'n_estimators': [5, 10, 50, 100],
    'max_depth': list(range(1, 50)),
    'class_weight': ['balanced', None],
    'criterion': ['entropy', 'gini']
}

In [16]:
rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=21),
    param_grid_rf,
    scoring='accuracy',
    n_jobs=-1,
    cv=5
)

In [17]:
rf_grid.fit(X_train, y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",RandomForestC...ndom_state=21)
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'class_weight': ['balanced', None], 'criterion': ['entropy', 'gini'], 'max_depth': [1, 2, ...], 'n_estimators': [5, 10, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and 

In [18]:
rf_results = pd.DataFrame(rf_grid.cv_results_)
rf_results = rf_results.sort_values('rank_test_score').filter(
    ['params', 'mean_test_score', 'std_test_score', 'rank_test_score']
)

In [19]:
print(f"Best RF Parameters: {rf_grid.best_params_}")
print(f"Best RF Accuracy: {rf_grid.best_score_:.5f}")
rf_results.head(10)

Best RF Parameters: {'class_weight': None, 'criterion': 'gini', 'max_depth': 28, 'n_estimators': 50}
Best RF Accuracy: 0.90429


,params,mean_test_score,std_test_score,rank_test_score
698,"{'class_weight': None, 'criterion': 'gini', 'm...",0.904290,0.010961,1
711,"{'class_weight': None, 'criterion': 'gini', 'm...",0.903547,0.014380,2
314,"{'class_weight': 'balanced', 'criterion': 'gin...",0.902817,0.013554,3
330,"{'class_weight': 'balanced', 'criterion': 'gin...",0.902809,0.013010,4
763,"{'class_weight': None, 'criterion': 'gini', 'm...",0.902806,0.010460,5
759,"{'class_weight': None, 'criterion': 'gini', 'm...",0.902806,0.010460,5
767,"{'class_weight': None, 'criterion': 'gini', 'm...",0.902806,0.010460,5
755,"{'class_weight': None, 'criterion': 'gini', 'm...",0.902806,0.010460,5
735,"{'class_weight': None, 'criterion': 'gini', 'm...",0.902806,0.010460,5
783,"{'class_weight': None, 'criterion': 'gini', 'm...",0.902806,0.010460,5


## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [25]:
n_estimators_list = [5, 10, 50, 100]
max_depth_list = list(range(1, 50))
class_weight_list = ['balanced', None]
criterion_list = ['entropy', 'gini']

parameter_combinations = list(itertools.product(
    n_estimators_list, max_depth_list, class_weight_list, criterion_list
))

results_list = []

In [26]:
print(f"Starting manual gridsearch for {len(parameter_combinations)} combinations...")

for n, d, cw, c in tqdm(parameter_combinations):
    model = RandomForestClassifier(
        n_estimators=n,
        max_depth=d,
        class_weight=cw,
        criterion=c,
        random_state=21,
        n_jobs=-1 
    )
    
    scores = cross_val_score(model, X_train, y_train, cv=5)
    
    results_list.append({
        'n_estimators': n,
        'max_depth': d,
        'class_weight': cw,
        'criterion': c,
        'mean_accuracy': scores.mean(),
        'std_accuracy': scores.std()
    })


Starting manual gridsearch for 784 combinations...


100%|██████████| 784/784 [04:45<00:00,  2.75it/s]


In [27]:
manual_results_df = pd.DataFrame(results_list)
manual_results_df = manual_results_df.sort_values(by='mean_accuracy', ascending=False)

manual_results_df.head(10)

,n_estimators,max_depth,class_weight,criterion,mean_accuracy,std_accuracy
503,50,28,None,gini,0.904290,0.010961
711,100,31,None,gini,0.903547,0.014380
509,50,30,balanced,gini,0.902817,0.013554
525,50,34,balanced,gini,0.902809,0.013010
743,100,39,None,gini,0.902806,0.010460
751,100,41,None,gini,0.902806,0.010460
731,100,36,None,gini,0.902806,0.010460
735,100,37,None,gini,0.902806,0.010460
759,100,43,None,gini,0.902806,0.010460
775,100,47,None,gini,0.902806,0.010460


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [28]:
best_n = 50
best_d = 28
best_cw = None
best_crit = 'gini'

In [29]:
final_model = RandomForestClassifier(
    n_estimators=best_n, 
    max_depth=best_d, 
    class_weight=best_cw, 
    criterion=best_crit, 
    random_state=21
)

In [30]:
final_model.fit(X_train, y_train)

y_pred = final_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)

print(f"Final accuracy on the test set: {test_accuracy:.5f}")

Final accuracy on the test set: 0.92899
